In [2]:
import torch
from torch import nn, optim
from torch.optim import Adam
import torch.utils.data as data
import math
from collections import Counter
import collections
import numpy as np
import copy
import time
import spacy
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import time


def bleu_stats(hypothesis, reference):
    stats = []
    stats.append(len(hypothesis))
    stats.append(len(reference))
    for n in range(1, 5):
        s_ngrams = Counter(
            [tuple(hypothesis[i:i + n]) for i in range(len(hypothesis) + 1 - n)]
        )
        r_ngrams = Counter(
            [tuple(reference[i:i + n]) for i in range(len(reference) + 1 - n)]
        )

        stats.append(max([sum((s_ngrams & r_ngrams).values()), 0]))
        stats.append(max([len(hypothesis) + 1 - n, 0]))
    return stats

def bleu(stats):
    if len(list(filter(lambda x: x == 0, stats))) > 0:
        return 0
    (c, r) = stats[:2]
    log_bleu_prec = sum(
        [math.log(float(x) / y) for x, y in zip(stats[2::2], stats[3::2])]
    ) / 4.
    return math.exp(min([0, 1 - float(r) / c]) + log_bleu_prec)


def get_bleu(hypotheses, reference):
    stats = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
    for hyp, ref in zip(hypotheses, reference):
        print(hyp, ref)
        print(np.array(bleu_stats(hyp, ref)))
        stats += np.array(bleu_stats(hyp, ref))

    return 100 * bleu(stats)


ref = ['đây', 'là']
hyp = ['đây', 'là']

print(get_bleu(hyp, ref))

đây đây
[3 3 3 3 2 2 1 1 0 0]
là là
[2 2 2 2 1 1 0 0 0 0]
0


In [3]:
ref = ['is', 'are']
hyp = ['is', 'are']
print(get_bleu(hyp, ref))

is is
[2 2 2 2 1 1 0 0 0 0]
are are
[3 3 3 3 2 2 1 1 0 0]
0


In [13]:


def bleu(pred_seq, label_seq, k = 4):
    """Compute the BLEU."""
    pred_tokens, label_tokens = pred_seq.split(' '), label_seq.split(' ')
    len_pred, len_label = len(pred_tokens), len(label_tokens)
    score = math.exp(min(0, 1 - len_label / len_pred))

    

    for n in range(1, min(k, len_pred) + 1):
        num_matches, label_subs = 0, collections.defaultdict(int)
        for i in range(len_label - n + 1):
            label_subs[' '.join(label_tokens[i: i + n])] += 1

        for i in range(len_pred - n + 1):
            if label_subs[' '.join(pred_tokens[i: i + n])] > 0:
                num_matches += 1
                label_subs[' '.join(pred_tokens[i: i + n])] -= 1
        
        #print(num_matches)

        score *= math.pow(num_matches / (len_pred - n + 1), math.pow(0.5, n))
    return score

ref = 'đây là một vi vi vi vi ví_dụ'
hyp = 'đây là một ví vi vi vi vi'

bleu(hyp, ref)

0.713111832839726